In [1]:
import os
import json
import requests

APIID = os.environ.get('DMM_API_ID', 'b7fkZaG3pW6ZZHpGBbLz')
AFFILIATEID = os.environ.get('DMM_AFFILIATE_ID', 'kamipanmen-990')

In [2]:
def genre_search():
    hits_count = 10
    offset_count = 1


    while True:
        genre_url = f'https://api.dmm.com/affiliate/v3/ItemList?api_id={APIID}&affiliate_id={AFFILIATEID}&site=FANZA&service=digital&floor=videoa&hits={hits_count}&sort=date&offset={offset_count}&article=genre&article_id=6940&output=json'
        response = requests.get(genre_url)
        genre_text = response.text
        genre_data = json.loads(genre_text)
        genre_item = genre_data['result']['items']


        for items in genre_item:
            try:
                af_url = items['affiliateURL']
                title = items['title']
                videos_info = items['sampleMovieURL']
                del videos_info['pc_flag'], videos_info['sp_flag']

                size_array = []
                video_array = []
                for size, v_url in videos_info.items():

                    max_size_info = size.split('_')
                    split_size = int(max_size_info[1])
                    size_array.append(split_size)
                    video_array.append(v_url)

                max_size = size_array.index(max(size_array))
                if str(size_array[max_size]) in v_url:

                    if videos_info:
                        yield dict(
                            title=title,
                            aff_url=af_url,
                            video_url=v_url
                        )
            except Exception as ex:
                print(ex)

        offset_count = hits_count + offset_count

        if len(genre_item) is 0:
            break


<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\PC_User\AppData\Local\Temp\ipykernel_23348\1794861392.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if len(genre_item) is 0:


In [ ]:
for i in genre_search():
    print(i)

In [4]:
videos = {
        'size_476_306': 'https:size_476_306',
        'size_560_360': 'https:size_644_414',
        'size_644_414': 'https:size_644_414',
        'size_720_480': 'https:size_720_480',
}

In [6]:
multiply = lambda x: (int(x.split('_')[1]) * int(x.split('_')[2]),x)
key = sorted([*map(multiply, videos.keys())], reverse=False)[0][1]

In [9]:
key

'size_476_306'

In [7]:
videos[key]

'https:size_476_306'

In [1]:
l = ['Mon', 'tuw', 'wed', 'Thu', 'fri', 'sat', 'Sun']

In [2]:
def chage(words, func):
    for word in words:
        print(func(word))

In [3]:
def sam_func(word):
    return word.capitalize()

In [4]:
chage(l, sam_func)

Mon
Tuw
Wed
Thu
Fri
Sat
Sun


### フォロー済みIDを取得

In [6]:
import tweepy
import api_togsi7 as API
import json
import random

import logging

logger = logging.getLogger("tweepy")
logger.setLevel(logging.DEBUG)
handler = logging.FileHandler(filename="tweepy.log")
logger.addHandler(handler)

def apicall(API):
    client = tweepy.Client(consumer_key=API.API_KEY, consumer_secret=API.API_SECRET, access_token=API.ACCESS_TOKEN, \
        access_token_secret=API.ACCESS_TOKEN_SECRET, bearer_token=API.Bearer_token, wait_on_rate_limit=True)
    return client

def followed_mine(client, my_id):
    """自分アカウントのフォローしてるIDを取得"""
    followed = client.get_users_following(id=my_id, user_fields=["id", "name"])
    follow_id_list = [follow.id for follow in followed.data]
    # no_follow_id = list(set(fid + new_fid))
    # print(no_follow_id)
    print(follow_id_list)
    print(len(follow_id_list))

    follow_dict: dict[str] = {'id': follow_id_list}
    print(follow_dict)
    return follow_dict

### JSONに保存する関数

In [18]:
def json_save(ids: dict[str], json_name):
    """フォロー or フォロワー候補IDをjson dump"""
    with open(f'./{json_name}.json', 'w', encoding='utf-8') as f:
        json.dump(ids, f, indent=4, ensure_ascii=False)

### フォローしてる人をJSON保存

In [ ]:
my_id = 1515696887781015558 #togsi ID
client = apicall(API)
follow_dict = followed_mine(client, my_id)
json_save(ids=follow_dict, json_name='following_id')

In [55]:
load = json.load(open('following_id.json'))
len(load['id'])

35

### 新しくフォローするIDを取得

In [16]:
def new_follow_id(client):
    """最新のツイートにいいねしているアカウントIDを取得する"""

    response = client.get_list_tweets(id=1514978714572173313, max_results=15,  expansions=["attachments.media_keys","referenced_tweets.id"])
    tweets = response.data
    random.shuffle(tweets)

    follow_id_lists = []
    max_count= 40

    for tweet in tweets:
        tid = tweet.id
        like_users = client.get_liking_users(tid) # TODO マックス40人までとかに制限する
        follows = like_users.data

        if follows:
            print(f'follows（フォローする人）が{len(like_users)}人いました')

            for follow in follows:
                follow_id_lists.append(follow.id)

    new_follow_dict: dict[str] = {'id': follow_id_lists[:max_count]}
    print(len(follow_id_lists), follow_id_lists[:max_count])
    return new_follow_dict

In [ ]:
client = apicall(API)
new_follow_dict = new_follow_id(client)
json_save(new_follow_dict, json_name='new_follow_id')

In [1]:
import ujson

In [89]:
following: json = ujson.load(open('following_id.json'))
following_list = following['id']
len(following['id'])

50

In [86]:
new_follow: json = ujson.load(open('./new_follow_id.json'))
new_follow_list = new_follow['id']
len(new_follow_list)

15

### 新しくフォローするIDだけ抽出

In [87]:
follow_list = list(set(new_follow_list)- set(following_list))
len(follow_list)

0

### 新フォロー実施

In [92]:
def follows(client, follow_list):
    like_user_follow = 0

    for id in follow_list:
        client.follow_user(target_user_id=id)
        print('フォローしました')
        like_user_follow += 1
        if like_user_follow >= 40:
            break

# フォローするとき#外す

In [ ]:
#follows(client, follow_list=follow_list)

### フォローしたらjsonへ保存する

In [67]:
for i in follow_list:
    following_list.append(i)

In [81]:
following_list = list(set(following_list)) #setはJSON保存できないのでリストにする
new_following = {}
new_following['id'] = following_list

In [82]:
json_save(ids=new_following, json_name='following_id')

In [94]:
__update__ = '20220507 17:51'

print(__update__)

20220507 17:51


### 自動化する前提でアンフォローも実装

# アンフォロー実行スクリプト -> 実行ファイル化してテストする

In [3]:
import auto_follow_unfollow_module
import api_togsi7 as API

client = auto_follow_unfollow_module.apicall(API)
my_id = 1515696887781015558 #togsi ID

### 最新のJSONにする
auto_follow_unfollow_module.follower_json_save(client=client, my_id=my_id)
unfollow_list = auto_follow_unfollow_module.unfollow_id_only()

### アンフォロー実施 ###
unfollow_done_list = auto_follow_unfollow_module.unfollows(client=client, unfollow_list=unfollow_list)


### アンフォローしたらJSONへ保存する
if unfollow_done_list:
    for i in unfollow_done_list:
        following_list.append(i)

    following_list = list(set(following_list)) #setはJSON保存できないのでリストにする
    new_following = {}
    new_following['id'] = following_list
    json_save(ids=new_following, json_name='following_id')


[1286592519199961088, 949822493287596033, 2279172336, 1465705811448000524, 1467401884025901059, 1473149795779104769, 1435567344923779072, 1513508550005256197, 1203932458263035905, 1068906813674872833, 1504416469370929161, 911586384334295040, 1465324484613664776, 1518566266919079936, 1517121214669418497, 1515878934671355907, 1471035446285471748, 1501237478514036742, 1158554927142035456, 1514977383216205834, 1515514246775582722, 1515697390480945160]
22
{'id': [1286592519199961088, 949822493287596033, 2279172336, 1465705811448000524, 1467401884025901059, 1473149795779104769, 1435567344923779072, 1513508550005256197, 1203932458263035905, 1068906813674872833, 1504416469370929161, 911586384334295040, 1465324484613664776, 1518566266919079936, 1517121214669418497, 1515878934671355907, 1471035446285471748, 1501237478514036742, 1158554927142035456, 1514977383216205834, 1515514246775582722, 1515697390480945160]}


In [6]:
import auto_follow_unfollow_module
import api_togsi7 as API

client = auto_follow_unfollow_module.apicall(API)
my_id = 1515696887781015558 #togsi ID

# フォロー実行スクリプト

In [ ]:

### 最新のJSONにする
auto_follow_unfollow_module.following_json_save(client=client, my_id=my_id)
auto_follow_unfollow_module.new_follow_json_save()

### フォロー実施 ###
follow_list = auto_follow_unfollow_module.new_follow_id_only()
follow_done_list = follows(client, follow_list)

### フォローしたらJSONへ保存する
if follow_done_list:
    for i in follow_done_list:
        following_list.append(i)

    following_list = list(set(following_list)) #setはJSON保存できないのでリストにする
    new_following = {}
    new_following['id'] = following_list
    json_save(ids=new_following, json_name='following_id')

In [3]:
import os
print(os.getcwd())

/Users/ore/Documents/GitHub/DMM/twitter_aff_videos/follow_auto
